In [ ]:
# from pip._internal import main as pipmain
# pipmain(['install','openpyxl'])
#use ! before pip
# !pip install scipy
# !pip install matplotlib

from tkinter import *

import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog

import tkinter.font as font
import pandas as pd
import numpy as np

import Factors as fc
import Utils as uti

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

## for Factors
from scipy.stats import rankdata
import heapq
###################################################
## Show all columns
pd.set_option("display.max.columns", None)
avs_matrix_values = np.zeros((4,4))

# real_bins = np.linspace(0, 1, 5+1)
# print("real_bins", real_bins)

## default input data
max_score = 1
bin_number = 5
threshold_percent = 0.6
pareto_percent = 0.25
factor_input_list = [max_score, bin_number, threshold_percent, pareto_percent]


In [ ]:
def initializeInputList(input_list, vars):
    print("next call")
    for i in range(len(vars)):
        if i <2:
            input_list[i] = input_list[i] if vars[i].get() =='' else int(vars[i].get())
        else:
            input_list[i] = input_list[i] if vars[i].get() =='' else float(vars[i].get())
            
        
def mainFunction(input_list, data_list, avs_matrix_values, i):
    for i in range(16):       
        list_name = uti.cell_name_list[i].replace('/', '_')    
        df = pd.read_excel("EconomicTable.xlsx", sheet_name=list_name)
        fact = fc.Factors(df, input_list)
        fact.calculate()
        answer = fact.table_value        
        dominant_factors =fact.dominant_factors_name_list       
        data_list[i] = (answer, dominant_factors)
        uti.calculate(data_list[i], i+1, avs_matrix_values)
        

def placeInputWidgets(input_list, data_list, avs_matrix_values, i, root):
    frame = Frame(root, highlightbackground = "#606060",  highlightthickness=2, height=350)
    frame.place(x = 45, y = 125)
    vars = []
    name_list = ["макс балл", "число столбцов", "порог%", "Паретто%"]
    for i in range(len(input_list)):
        var = StringVar()
        lbl = Label(frame, text = name_list[i])
        lbl.grid(row=0, column=i, sticky=NSEW)
        text_entry = Entry(frame, textvariable=var)
        text_entry.grid(row=1, column=i, sticky=NSEW)
        vars.append(var)
    btn = Button(frame, text="Рассчитать", command=lambda:[initializeInputList(input_list, vars),
                                mainFunction(input_list, data_list, avs_matrix_values, i)])
    btn.grid(row=1, column=len(input_list), sticky=NSEW)
    
   
    

    

In [ ]:
root = Tk()
root.iconbitmap("matrix.ico")
# photo = PhotoImage(file = '/home/arseniy/Desktop/FactorsLevels/matrix.png')
# root.wm_iconphoto(False, photo)
# root.img = tk.PhotoImage(file='/home/arseniy/Desktop/FactorsLevels/matrix.png')
# root.iconphoto( False, root.img ) # application icon

root.title("Матрица 4x4 AVS")
label=Label(root, text="AVS матрица: ", font=("Times New Roman", 45, "bold"), width=22, height=2).pack()
root.geometry('1600x900+100+60')
frame = Frame(root)
frame.place(x=50, y=200)
 
uti.createTableLabels(frame)     

data_list = []
btn_list = []
    
for i in range(16):       
        list_name = uti.cell_name_list[i].replace('/', '_')    
        df = pd.read_excel("EconomicTable.xlsx", sheet_name=list_name)
        fact = fc.Factors(df, factor_input_list)
        fact.calculate()
        answer = fact.table_value        
        dominant_factors =fact.dominant_factors_name_list
        
        data_list.append((answer, dominant_factors)) 
        
###Без лямбды будет сразу срабатывать 
        btn = Button( frame, text = uti.cell_name_list[i], width=20,
              disabledforeground="black",
              command=lambda i=i: uti.buttonFunction(data_list[i], root) ) 
        btn['font'] = font.Font(family='Helvetica', size=14)
        
        x = int(i)%4
        y = int(i)//4
        btn.grid(row=y+1, column=x+1,sticky=NSEW)
        btn_list.append(btn)
        uti.calculate(data_list[i], i+1, avs_matrix_values)
        
#### input widgets
placeInputWidgets(factor_input_list,data_list, avs_matrix_values, i, root)       
##########################################################

#### find nonsistematic coefficient value
Rsist=10.4
IRR = 30.99
risks = fc.Risks(Rsist, IRR)
risks.calculate(avs_matrix_values)

uti.showInputOutput(risks, root, avs_matrix_values)
uti.placeGraph(risks, root)
     
button = Button(root, text="full matrix", width=20, command=lambda:
                uti.fullMatrix(root, avs_matrix_values, risks))
button.pack( side = BOTTOM, pady=0, ipadx=10)
button = Button(root, text="Показать значения", width=20, command=lambda: uti.changeButtonText(btn_list, avs_matrix_values))
button.place(x=50, y=400)

# print("avs ", avs_matrix_values)

root.mainloop()
                                                            